#### (Home credit competition 중심으로)
## 2.1 데이터 정제하기-레이블/원 핫 인코딩

---
### 소스 및 데이터는 아래 kaggle을 참조하였습니다. 
* 캐글 주소 : https://www.kaggle.com/willkoehrsen/start-here-a-gentle-introduction
* 해당 커널의 한글 번역은 파파고와 구글을 활용하였습니다.
___

In [1]:
# 라이브러리 
import pandas as pd
import numpy as np
import os

from sklearn.preprocessing import LabelEncoder
import warnings
warnings.filterwarnings('ignore')

import matplotlib.pyplot as plt
import seaborn as sns

%matplotlib inline 

In [2]:
# 데이터 셋
app_train = pd.read_csv('d:/Projects/Cp/input/application_train.csv')
app_test = pd.read_csv('d:/Projects/Cp/input/application_test.csv')

### 컬럼의 데이터형 확인 

* 학습용 데이터 특정 컬럼 데이터형 확인

In [3]:
app_train['TARGET'].dtype

dtype('int64')

* 학습용 데이터 전체 열(컬럼) 데이터형 확인

In [4]:
print(app_train.dtypes)

SK_ID_CURR                      int64
TARGET                          int64
NAME_CONTRACT_TYPE             object
CODE_GENDER                    object
FLAG_OWN_CAR                   object
FLAG_OWN_REALTY                object
CNT_CHILDREN                    int64
AMT_INCOME_TOTAL              float64
AMT_CREDIT                    float64
AMT_ANNUITY                   float64
AMT_GOODS_PRICE               float64
NAME_TYPE_SUITE                object
NAME_INCOME_TYPE               object
NAME_EDUCATION_TYPE            object
NAME_FAMILY_STATUS             object
NAME_HOUSING_TYPE              object
REGION_POPULATION_RELATIVE    float64
DAYS_BIRTH                      int64
DAYS_EMPLOYED                   int64
DAYS_REGISTRATION             float64
DAYS_ID_PUBLISH                 int64
OWN_CAR_AGE                   float64
FLAG_MOBIL                      int64
FLAG_EMP_PHONE                  int64
FLAG_WORK_PHONE                 int64
FLAG_CONT_MOBILE                int64
FLAG_PHONE  

In [5]:
# 열의 각 유형 수
app_train.dtypes.value_counts()

float64    65
int64      41
object     16
dtype: int64

데이터의 유형

* int64, float64는 숫자 변수(이산 또는 연속형) 106개
* 객체(object)열은 문자열 포함하며 범주형 : 16개

In [6]:
# 범주형 열(컬럼)의 고유 항목 수 내림차순 정렬
app_train.select_dtypes('object').apply(pd.Series.nunique, axis = 0).sort_values(ascending=False)

ORGANIZATION_TYPE             58
OCCUPATION_TYPE               18
NAME_INCOME_TYPE               8
WALLSMATERIAL_MODE             7
WEEKDAY_APPR_PROCESS_START     7
NAME_TYPE_SUITE                7
NAME_HOUSING_TYPE              6
NAME_FAMILY_STATUS             6
NAME_EDUCATION_TYPE            5
FONDKAPREMONT_MODE             4
HOUSETYPE_MODE                 3
CODE_GENDER                    3
EMERGENCYSTATE_MODE            2
FLAG_OWN_REALTY                2
FLAG_OWN_CAR                   2
NAME_CONTRACT_TYPE             2
dtype: int64

* 범주형 열은 비교적 적은 수의 고유 항목을 가지고 있음을 알 수 있다.
* 범주형 고유항목에 널(Null)은 미포함

#### 레이블 인코딩
기계학습 모델은 LightGBM과 같은 일부 모델을 제외하고 범주형 변수를 처리할 수 없습니다. 따라서 범주형 변수를 모델로 전달하기 전에 숫자로 표현(인코딩) 하는 방법을 찾아야 합니다.

* 레이블 인코딩 : 각 범주형 값에 숫자 지정

|ex)|범주형|숫자|
|:---:|:---:|:---:|
|0|양말|0|
|1|여우|1|
|2|양말|2|
|3|상자|1|

* 새로운 열이 추가되지 않습니다.
* 범주에 지정된 숫자는 임의이며 범주의 고유한 특성을 반영하지 않습니다.
* 그러나 기계학습 모델은 범주형에 지정된 숫자값에 가중치를 할 당하여 모델에 영향을 줄수 있습니다. 
* 따라서 범주형 값(남자/여자)이 두가지 고유한 값만 있는 경우에는 레이블 인코딩이 문제가 되지 않지만 두가지 이상의 범주형 값을 가진다면 원 핫 인코딩이 안전합니다.
---

* 원 핫 인코딩

|ex)|범주형|범주형특성_상자|범주형특성_여우|범주형특성_양말|
|:---:|:---:|:---:|:---:|:---:|
|0|양말|0|0|1|
|1|여우|0|1|0|
|2|양말|0|0|1|
|3|상자|1|0|0|

* 원 핫 인코딩의 단점은 범주형 변수로 데이터의 크기가 늘어나는 것입니다.
* 이를 처리하기 위해 PCA, 차원 감소 방법으로 차원 수를 줄입니다.

[ 참고자료 ] 파이썬 파이브러리를 활용한 머신러닝 266페이지, 안드레아스 뮐러, 세라 가이도 지음, 한빛미디어, 2017)

#### 해당 데이터의 범주형 처리
* 2개 범주형 변수 : 레이블인코딩 사용
* 2개 이상 범부형 변수 : 원 핫 인코딩 사용

In [7]:
# 데이터형이 오브젝트이면서 고유항목수가 2이하인 열

for col in app_train:
    if app_train[col].dtype == 'object':
        if len(list(app_train[col].unique())) <= 2:
            print(col)

NAME_CONTRACT_TYPE
FLAG_OWN_CAR
FLAG_OWN_REALTY


위에서 고유항목 수가 2인 열은 다음과 같이 4개이나

1. EMERGENCYSTATE_MODE            
2. FLAG_OWN_REALTY               
3. FLAG_OWN_CAR                  
4. NAME_CONTRACT_TYPE             

실제로는 3개 열만 해당이 됨

1. NAME_CONTRACT_TYPE
2. FLAG_OWN_CAR
3. FLAG_OWN_REALTY

In [8]:
print("app_train 데이터의 총 행 : ", app_train.shape[0])
print("EMERGENCYSTATE_MODE 열의 결측값 수 : ", app_train['EMERGENCYSTATE_MODE'].isnull().sum())
print("EMERGENCYSTATE_MODE 열의 수 : ", app_train['EMERGENCYSTATE_MODE'].isnull().sum())
print("--------------------------------------------------------------------")
print("EMERGENCYSTATE_MODE 범주")
app_train['EMERGENCYSTATE_MODE'].value_counts()

app_train 데이터의 총 행 :  307511
EMERGENCYSTATE_MODE 열의 결측값 수 :  145755
EMERGENCYSTATE_MODE 열의 수 :  145755
--------------------------------------------------------------------
EMERGENCYSTATE_MODE 범주


No     159428
Yes      2328
Name: EMERGENCYSTATE_MODE, dtype: int64

* 즉 EMERGENCYSTATE_MODE 열은 다음과 같이 3개의 항목을 가지고 있고 apply(pd.Series.nunique, axis = 0) 처리 시 Null값은 제외하고 구하므로 실제 범주항목은 3이나 2로 구해졌다 

|범주값|갯수|
|:---:|---:|
|No|159428|
|Yes|2328|
|Null|145755|

In [9]:
# 레이블 인코딩 전 데이터 크기
print('after app_train Features shape : ', app_train.shape)
print('after app_test Features shape : ', app_test.shape) 

after app_train Features shape :  (307511, 122)
after app_test Features shape :  (48744, 121)


In [10]:
# 레이블 인코딩 전 데이터 범주 항목
print(app_train['NAME_CONTRACT_TYPE'].value_counts())
print("--------------------------------------------")
print(app_train['FLAG_OWN_CAR'].value_counts())
print("--------------------------------------------")
print(app_train['FLAG_OWN_REALTY'].value_counts())

Cash loans         278232
Revolving loans     29279
Name: NAME_CONTRACT_TYPE, dtype: int64
--------------------------------------------
N    202924
Y    104587
Name: FLAG_OWN_CAR, dtype: int64
--------------------------------------------
Y    213312
N     94199
Name: FLAG_OWN_REALTY, dtype: int64


In [11]:
#  레이블 인코딩 
le = LabelEncoder()
le_count = 0

for col in app_train:    
    if app_train[col].dtype == 'object':
        if len(list(app_train[col].unique())) <= 2: 
            print(col)
            le.fit(app_train[col])
            app_train[col] = le.transform(app_train[col])
            app_test[col]  = le.transform(app_test [col])            
            le_count += 1
            
print('%d columns were label encoded.' % le_count)

NAME_CONTRACT_TYPE
FLAG_OWN_CAR
FLAG_OWN_REALTY
3 columns were label encoded.


In [12]:
# 레이블 인코딩 후 데이터 형태
print('before app_train Features shape : ', app_train.shape)
print('before app_test Features shape : ', app_test.shape)

before app_train Features shape :  (307511, 122)
before app_test Features shape :  (48744, 121)


In [13]:
# 레이블 인코딩 후 데이터 범주 항목
print(app_train['NAME_CONTRACT_TYPE'].value_counts())
print("--------------------------------------------")
print(app_train['FLAG_OWN_CAR'].value_counts())
print("--------------------------------------------")
print(app_train['FLAG_OWN_REALTY'].value_counts())

0    278232
1     29279
Name: NAME_CONTRACT_TYPE, dtype: int64
--------------------------------------------
0    202924
1    104587
Name: FLAG_OWN_CAR, dtype: int64
--------------------------------------------
1    213312
0     94199
Name: FLAG_OWN_REALTY, dtype: int64


* 레이블 인코딩은 열의 추가가 없고 범주형 항목이 1/0으로 바뀌는 것을 확인 할 수 있다.

In [14]:
# 원 핫 인코딩 전 데이터 크기
print('after app_train Features shape : ', app_train.shape)
print('after app_test Features shape : ', app_test.shape) 

after app_train Features shape :  (307511, 122)
after app_test Features shape :  (48744, 121)


In [15]:
# 원 핫 인코딩
app_train = pd.get_dummies(app_train)
app_test = pd.get_dummies(app_test)

In [16]:
# 원 핫 인코딩 후 데이터 크기
print('after app_train Features shape : ', app_train.shape)
print('after app_test Features shape : ', app_test.shape) 

after app_train Features shape :  (307511, 243)
after app_test Features shape :  (48744, 239)


* 원 핫 인코딩 후 열의 수가 늘어난 것을 확인 할 수 있다.

#### 학습용 데이터와 테스트용 데이터 열(컬럼) 맞추기
* 원 핫 인코딩 후 학습용 데이터와 테스트용 데이터의 열(컬럼)의 갯수를 보면 서로 불일치
* 학습용 데이터에만 있고 테스트용 데이터에는 없는 TARGET(대출금 상환여부)열을 감안하더라도 학습용 데이터가 테스트용 데이터 보다 열(컬럼)이 3개가 더 많다.
* 이를 맞추는 작업을 한다.
---

* 학습용 데이터의 TARGET(대출금 상환여부)열을 따로 보관 후 학습용 데이터와 테스트용 데이터를 열 기준으로 정렬하여 서로 없는 열(컬럼) 삭제
* 따로 보관한 TARGET(대출금 상환여부)열을 학습용 데이터에 추가

In [17]:
# TARGET(대출금 상환여부)열 따로 보관
train_labels = app_train['TARGET']

# 열(컬럼)기준 데이터 정렬하여 서로 없는 열(컬럼) 삭제
app_train, app_test = app_train.align(app_test, join = 'inner', axis = 1)

# 학습용 데이터에 TARGET(대출금 상환여부)열 추가
app_train['TARGET'] = train_labels

print('Training Features shape: ', app_train.shape)
print('Testing Features shape: ', app_test.shape)

Training Features shape:  (307511, 240)
Testing Features shape:  (48744, 239)
